[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/alessandronascimento/BioMolComp/blob/main/P05/AnaliseFilogenetica3.ipynb)

# Introdução #

Obviamente, também podemos usar ferramentas do [Biopython](http://biopython.org) para análises filogenéticas.

O Biopython ainda não é muito completo para esta finalidade. Mas algumas análises preliminares podem ser realizadas rapidamente com as ferramentas do Biopython.

In [ ]:
#@title Preparando o Ambiente para a Execução

# Installing Biopython and importing required libraries...

!pip3 install biopython
from Bio import SeqIO
from Bio.Align.Applications import ClustalwCommandline
from Bio import Phylo
import os

# Gettting ClustalW2....

!wget http://www.clustal.org/download/current/clustalw-2.1-linux-x86_64-libcppstatic.tar.gz
!tar -xzf clustalw-2.1-linux-x86_64-libcppstatic.tar.gz
!mv clustalw-2.1-linux-x86_64-libcppstatic/clustalw2 .


# Análise Filogenética #

Nesta seção, vamos fazer o download da sequência de domínios transcriptase reversa de retrotransposons do tipo non-LTR que já usamos anteriormente. 

Na sequência inspecionaremos as sequências presentes neste arquivo.

In [ ]:
seq_file="filogenia1.fasta"

!wget https://raw.githubusercontent.com/alessandronascimento/BioMolComp/main/P05/filogenia1.fasta

count=0
for seq_record in SeqIO.parse(seq_file, "fasta"):
    count = count+1
    print("SeqID: %s" % seq_record.id)
    print(repr(seq_record.seq))
    print("Sequence Length: %d" % len(seq_record))
print("")    
print("Number of sequences found: %d" % count);


## Alinhamento das Sequências ##

Nesta etapa, usamos o programa [ClustalW](http://www.clustal.org/clustal2/) para preparar um alinhamento múltiplo de sequências (MSA). Como o ClustalW utiliza o método progressivo para o alinhamento múltiplo, as distâncias entre as sequências já são calculadas e salvas juntamente com o alinhamento. Efetivamente, o programa gera um arquivo:

```
to store a dendrogram that describes the similarity
of the sequences to each other.
```

Este arquivo **não é uma árvore filogenética**. Ele é a árvore guia (*guide-tree*) para a construção do alinhamento múltiplo. Mas, como o texto acima nos diz, ele agrupa as sequências similares, com as distâncias entre elas. Isto pode dar uma ideia do quão próximas as sequências são dentro de um alinhamento múltiplo.

Na etapa abaixo, vamos fazer o alinhamento múltiplo (MSA) com o clustalw.


In [ ]:
clustalw_exe = r"./clustalw2"
clustalw_cline = ClustalwCommandline(clustalw_exe, infile=seq_file, gapopen=5, gapext=0.1)
assert os.path.isfile(clustalw_exe), "Clustal W executable missing"
stdout, stderr = clustalw_cline()

from Bio import AlignIO
align = AlignIO.read("filogenia1.aln", "clustal")
for record in align:
  print("%s - %s" % (record.seq, record.id))


## Análise da Árvore Guia ##

O programa ClustalW deve ter gerado dois arquivos. O arquivo filogenia1.aln foi o arquivo que visualizamos na célula acima. Este arquivo contém o alinhamento gerado pelo método progressivo.

Já o arquivo filogenia1.dnd contem o dendograma com as distâncias (ou similaridades) que as sequências têm entre si. Este arquivo pode ser usado diretamente para a visualização de uma árvore de distâncias.

In [ ]:
tree = Phylo.read("filogenia1.dnd", "newick")
Phylo.draw_ascii(tree)

## Análise da Árvore ##

No método acima, representamos uma árvore usando um formato de desenho em texto (ASCII). Esta metodologia é simples e rápida e os braços da árvore têm distâncias proporcionais à similaridade entre as sequências.

É possível também gerar uma imagem com a árvore, desde que a biblioteca matplotlib esteja instalada e disponível (ela usualmente está disponível no Google Colab).

In [ ]:
tree.rooted = True
Phylo.draw(tree)

Compare a árvore obtida nesta análise com a árvore obtida na análise filogenética realizada com o MEGA11. Elas são similares? 